In [3]:
import torch
from transformers import BertTokenizer, BertModel, pipeline

def text_embedding(data):
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
    bert_model = BertModel.from_pretrained('bert-base-uncased').to(device)

    def get_bert_embeddings(data):
        tokens = tokenizer(data.tolist(), padding=True, truncation=True, return_tensors='pt').to(device)
        with torch.no_grad():
            embeddings = bert_model(**tokens).last_hidden_state.mean(dim=1)
        return embeddings

    batch_size = 128
    num_samples = len(data)
    num_batches = (num_samples + batch_size - 1) // batch_size

    embeddings_list = []

    for i in range(num_batches):
        start_idx = i * batch_size
        end_idx = (i + 1) * batch_size
        batch_data = data.iloc[start_idx:end_idx]
        batch_embeddings = get_bert_embeddings(batch_data)
        embeddings_list.append(batch_embeddings)

    embeddings = torch.cat(embeddings_list, dim=0).cpu().numpy()
    return embeddings

In [2]:
import pandas as pd

df = pd.read_csv('../original_files/politifact_data_2022_score.csv')

df['documented_time'] = pd.to_datetime(df['documented_time'])

df = df[~df['label'].isin({'full-flop', 'half-flip', 'no-flip'})]

#only rows after 2022 (recent)
df = df[df['documented_time'].dt.year >= 2022]

label_map = {'pants-fire': 5, 'false': 4, 'barely-true': 3, 
             'half-true': 2, 'mostly-true': 1, 'true': 0}
df['label'] = df['label'].astype(int)
df['label'] = df['label'].replace(label_map)

In [1]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction.text import ENGLISH_STOP_WORDS
from gnews import GNews
import numpy as np
from langchain.text_splitter import CharacterTextSplitter
from langchain_community.document_loaders import TextLoader
from langchain_community.vectorstores import Weaviate
import nltk
import torch
from transformers import BertTokenizer, BertModel, pipeline
nltk.download('punkt')
def rag(content):
    # Advance RAG
    print("Retrieving keywords...")
    tfidf_vectorizer = TfidfVectorizer(stop_words=ENGLISH_STOP_WORDS)
    tfidf_matrix = tfidf_vectorizer.fit_transform([content])
    feature_names = tfidf_vectorizer.get_feature_names()
    scores = tfidf_matrix.toarray().flatten()
    indices = scores.argsort()[::-1]
    top_n = 10
    top_features = [(feature_names[i], scores[i]) for i in indices[:top_n]]
    keywords = " ".join([feature for feature, score in top_features])
    for feature, score in top_features:
        print(f"{feature}: {score}")
    print("RAG: Getting new evidence...")
    google_news = GNews()
    max_results = 4
    # google_news.period = '7d'
    google_news.max_results = max_results 
    # google_news.country = 'United States'
    google_news.language = 'english'
    # google_news.exclude_websites = ['yahoo.com', 'cnn.com'] 
    google_news.start_date = (2022, 1, 1)
    google_news.end_date = (2023, 12, 31)
    articles = []
    news = google_news.get_news(keywords)
    for i in range(max_results):
        try:
            article = google_news.get_full_article(
                news[i]['url']
            )
        except:
            break
        articles.append(article)
    title_text = [article.title for article in articles if article]
    article_text = [article.text for article in articles if article]

    # Chunk the google news
    class Document:
        def __init__(self, text):
            self.page_content = text
            self.metadata = {'source': 'google news'}

    print("Chunking the articles")
    documents = [Document(article) for article in article_text]
    text_splitter = CharacterTextSplitter(chunk_size=300, chunk_overlap=0)
    chunked_articles = text_splitter.split_documents(documents)
    chunked_articles = [document.page_content for document in chunked_articles]
    
    # #Advance RAG
    print("Posting new evidence to vector database...")
    for article in chunked_articles:
        # Check for duplicate before posting
        try:
            properties = {"context": article}
            vector = text_embedding(pd.Series(article)).tolist()[0]
            client.data_object.create(properties, "test_dataset_1", vector=vector)
        except:
            continue

2024-03-06 06:41:45.688919: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
03/06/2024 06:41:47 AM - Note: NumExpr detected 40 cores but "NUMEXPR_MAX_THREADS" not set, so enforcing safe limit of 8.
03/06/2024 06:41:47 AM - NumExpr defaulting to 8 threads.
[nltk_data] Downloading package punkt to /home/zhj003/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [4]:
import weaviate
client = weaviate.Client(
    url = "https://testing-cluster-2qgcoz4q.weaviate.network",  # Replace with your endpoint
    auth_client_secret=weaviate.auth.AuthApiKey(api_key="qRarwGLC0CwrpQsSpK64E1V0c3HajFoAy893"),  # Replace w/ your Weaviate instance API key
)

/home/zhj003/.local/lib/python3.9/site-packages/weaviate/warnings.py:158: DeprecationWarning: Dep016: You are using the Weaviate v3 client, which is deprecated.
            Consider upgrading to the new and improved v4 client instead!
            See here for usage: https://weaviate.io/developers/weaviate/client-libraries/python
            
  warnings.warn(


In [ ]:
from IPython.display import clear_output
for index, row in df.iterrows():
    client = weaviate.Client(
    url = "https://testing-cluster-2qgcoz4q.weaviate.network",  # Replace with your endpoint
        auth_client_secret=weaviate.auth.AuthApiKey(api_key="qRarwGLC0CwrpQsSpK64E1V0c3HajFoAy893"),  # Replace w/ your Weaviate instance API key
    )
    if index < 50:
        continue
    if index % 50 == 0:
        clear_output(wait=True)
        print(f"Running at iteration {index}")
    rag(row['article'])

Running at iteration 700
Retrieving keywords...
pentagon: 0.3491282676376715
facebook: 0.2493773340269082
attack: 0.2493773340269082
explosion: 0.2493773340269082
news: 0.19950186722152657
post: 0.19950186722152657
22: 0.19950186722152657
near: 0.19950186722152657
said: 0.14962640041614492
smoke: 0.14962640041614492
RAG: Getting new evidence...


03/07/2024 05:34:57 AM - Created a chunk of size 326, which is longer than the specified 300
03/07/2024 05:34:57 AM - Created a chunk of size 361, which is longer than the specified 300
03/07/2024 05:34:57 AM - Created a chunk of size 366, which is longer than the specified 300
03/07/2024 05:34:57 AM - Created a chunk of size 332, which is longer than the specified 300
03/07/2024 05:34:57 AM - Created a chunk of size 376, which is longer than the specified 300
03/07/2024 05:34:57 AM - Created a chunk of size 320, which is longer than the specified 300
03/07/2024 05:34:57 AM - Created a chunk of size 364, which is longer than the specified 300
03/07/2024 05:34:57 AM - Created a chunk of size 305, which is longer than the specified 300
03/07/2024 05:34:57 AM - Created a chunk of size 340, which is longer than the specified 300


Chunking the articles
Posting new evidence to vector database...
Retrieving keywords...
pandemic: 0.3731033238480316
said: 0.3497843661075296
flu: 0.2798274928860237
children: 0.2798274928860237
bird: 0.1865516619240158
avian: 0.16323270418351382
health: 0.13991374644301185
positive: 0.13991374644301185
tested: 0.13991374644301185
morris: 0.13991374644301185
RAG: Getting new evidence...
Chunking the articles
Posting new evidence to vector database...
Retrieving keywords...
bowman: 0.44992127066584753
killed: 0.22496063533292376
dr: 0.179968508266339
burdock: 0.179968508266339
sebi: 0.179968508266339
facebook: 0.13497638119975425
false: 0.13497638119975425
video: 0.13497638119975425
died: 0.13497638119975425
prison: 0.13497638119975425
RAG: Getting new evidence...


03/07/2024 05:36:29 AM - Created a chunk of size 456, which is longer than the specified 300
03/07/2024 05:36:29 AM - Created a chunk of size 437, which is longer than the specified 300


Chunking the articles
Posting new evidence to vector database...
Retrieving keywords...
election: 0.3916103699798974
lake: 0.3916103699798974
signatures: 0.24920659907811654
ballots: 0.23140612771539393
maricopa: 0.1958051849899487
mail: 0.1780047136272261
county: 0.1780047136272261
arizona: 0.1780047136272261
signature: 0.1602042422645035
verification: 0.14240377090178089
RAG: Getting new evidence...


...

Chunking the articles
Posting new evidence to vector database...
Retrieving keywords...
que: 0.4528120578009407
la: 0.3937496154790789
el: 0.3149996923832631
en: 0.29531221160930915
los: 0.19687480773953944
frontera: 0.15749984619163154
42: 0.15749984619163154
título: 0.15749984619163154
del: 0.1378123654176776
unidos: 0.1378123654176776
RAG: Getting new evidence...
Chunking the articles
Posting new evidence to vector database...
Retrieving keywords...
image: 0.36178730264621084
photo: 0.36178730264621084
ai: 0.28942984211696865
trump: 0.2170723815877265
images: 0.14471492105848432
showed: 0.14471492105848432
hands: 0.14471492105848432
generators: 0.14471492105848432
generated: 0.14471492105848432
october: 0.14471492105848432
RAG: Getting new evidence...


03/07/2024 06:44:22 AM - Created a chunk of size 302, which is longer than the specified 300
03/07/2024 06:44:22 AM - Created a chunk of size 433, which is longer than the specified 300
03/07/2024 06:44:22 AM - Created a chunk of size 318, which is longer than the specified 300
03/07/2024 06:44:22 AM - Created a chunk of size 419, which is longer than the specified 300
03/07/2024 06:44:22 AM - Created a chunk of size 516, which is longer than the specified 300
03/07/2024 06:44:22 AM - Created a chunk of size 368, which is longer than the specified 300
03/07/2024 06:44:22 AM - Created a chunk of size 320, which is longer than the specified 300
03/07/2024 06:44:22 AM - Created a chunk of size 318, which is longer than the specified 300
03/07/2024 06:44:22 AM - Created a chunk of size 509, which is longer than the specified 300
03/07/2024 06:44:22 AM - Created a chunk of size 479, which is longer than the specified 300
03/07/2024 06:44:22 AM - Created a chunk of size 827, which is longer 

Chunking the articles
Posting new evidence to vector database...
Retrieving keywords...
latino: 0.4130921946615823
businesses: 0.3951316644589048
owned: 0.2514474228374849
report: 0.2155263624321299
venture: 0.2155263624321299
said: 0.1975658322294524
business: 0.1975658322294524
capital: 0.16164477182409742
new: 0.16164477182409742
data: 0.16164477182409742
RAG: Getting new evidence...


03/07/2024 06:46:41 AM - Created a chunk of size 309, which is longer than the specified 300
03/07/2024 06:46:41 AM - Created a chunk of size 353, which is longer than the specified 300
03/07/2024 06:46:41 AM - Created a chunk of size 307, which is longer than the specified 300
03/07/2024 06:46:41 AM - Created a chunk of size 359, which is longer than the specified 300
03/07/2024 06:46:41 AM - Created a chunk of size 350, which is longer than the specified 300
03/07/2024 06:46:41 AM - Created a chunk of size 425, which is longer than the specified 300
03/07/2024 06:46:41 AM - Created a chunk of size 412, which is longer than the specified 300
03/07/2024 06:46:41 AM - Created a chunk of size 338, which is longer than the specified 300
03/07/2024 06:46:41 AM - Created a chunk of size 388, which is longer than the specified 300
03/07/2024 06:46:41 AM - Created a chunk of size 390, which is longer than the specified 300
03/07/2024 06:46:41 AM - Created a chunk of size 362, which is longer 

Chunking the articles
Posting new evidence to vector database...
Retrieving keywords...
biden: 0.464139560816457
child: 0.464139560816457
photo: 0.36099743619057767
shows: 0.154713186938819
inappropriately: 0.154713186938819
altered: 0.154713186938819
october: 0.154713186938819
post: 0.10314212462587934
facebook: 0.10314212462587934
flag: 0.10314212462587934
RAG: Getting new evidence...
